In [2]:
#setup and load
import pandas as pd
import numpy as np

# Upload CSV from local
from google.colab import files
uploaded = files.upload()

# Load it
df = pd.read_csv("first_25000_rows.csv")

# Check structure
print("Columns:", df.columns.tolist())
df.head()


Saving first_25000_rows.csv to first_25000_rows (1).csv
Columns: ['ts_recv', 'ts_event', 'rtype', 'publisher_id', 'instrument_id', 'action', 'side', 'depth', 'price', 'size', 'flags', 'ts_in_delta', 'sequence', 'bid_px_00', 'ask_px_00', 'bid_sz_00', 'ask_sz_00', 'bid_ct_00', 'ask_ct_00', 'bid_px_01', 'ask_px_01', 'bid_sz_01', 'ask_sz_01', 'bid_ct_01', 'ask_ct_01', 'bid_px_02', 'ask_px_02', 'bid_sz_02', 'ask_sz_02', 'bid_ct_02', 'ask_ct_02', 'bid_px_03', 'ask_px_03', 'bid_sz_03', 'ask_sz_03', 'bid_ct_03', 'ask_ct_03', 'bid_px_04', 'ask_px_04', 'bid_sz_04', 'ask_sz_04', 'bid_ct_04', 'ask_ct_04', 'bid_px_05', 'ask_px_05', 'bid_sz_05', 'ask_sz_05', 'bid_ct_05', 'ask_ct_05', 'bid_px_06', 'ask_px_06', 'bid_sz_06', 'ask_sz_06', 'bid_ct_06', 'ask_ct_06', 'bid_px_07', 'ask_px_07', 'bid_sz_07', 'ask_sz_07', 'bid_ct_07', 'ask_ct_07', 'bid_px_08', 'ask_px_08', 'bid_sz_08', 'ask_sz_08', 'bid_ct_08', 'ask_ct_08', 'bid_px_09', 'ask_px_09', 'bid_sz_09', 'ask_sz_09', 'bid_ct_09', 'ask_ct_09', 'symbol']

,ts_recv,ts_event,rtype,publisher_id,instrument_id,action,side,depth,price,size,...,ask_sz_08,bid_ct_08,ask_ct_08,bid_px_09,ask_px_09,bid_sz_09,ask_sz_09,bid_ct_09,ask_ct_09,symbol
0,2024-10-21T11:54:29.221230963Z,2024-10-21T11:54:29.221064336Z,10,2,38,C,B,1,233.62,2,...,155,1,7,233.25,234.13,55,400,2,1,AAPL
1,2024-10-21T11:54:29.223936626Z,2024-10-21T11:54:29.223769812Z,10,2,38,A,B,0,233.67,2,...,155,1,7,233.25,234.13,55,400,2,1,AAPL
2,2024-10-21T11:54:29.225196809Z,2024-10-21T11:54:29.225030400Z,10,2,38,A,B,0,233.67,3,...,155,1,7,233.25,234.13,55,400,2,1,AAPL
3,2024-10-21T11:54:29.712600612Z,2024-10-21T11:54:29.712434212Z,10,2,38,A,B,2,233.52,200,...,155,1,7,233.25,234.13,55,400,2,1,AAPL
4,2024-10-21T11:54:29.764839221Z,2024-10-21T11:54:29.764673165Z,10,2,38,C,B,2,233.52,200,...,155,1,7,233.25,234.13,55,400,2,1,AAPL


In [4]:
#Compute Best-Level OFI
import pandas as pd
import numpy as np

# Ensure proper datetime format
df['ts_event'] = pd.to_datetime(df['ts_event'])

# Filter for depth = 0 (best level) and symbol = AAPL
best_level = df[(df['depth'] == 0) & (df['symbol'] == 'AAPL')].copy()
best_level = best_level.sort_values('ts_event').reset_index(drop=True)

# Define OFI function per Cont et al. (2014)
def compute_ofi(curr, prev):
    if curr['side'] == 'B':
        if curr['price'] > prev['price']:
            return curr['size']
        elif curr['price'] == prev['price']:
            return curr['size'] - prev['size']
        else:
            return -curr['size']
    elif curr['side'] == 'A':
        if curr['price'] < prev['price']:
            return curr['size']
        elif curr['price'] == prev['price']:
            return prev['size'] - curr['size']
        else:
            return -curr['size']
    return 0

# Compute OFI for each event
ofi_values = [0]
for i in range(1, len(best_level)):
    ofi = compute_ofi(best_level.loc[i], best_level.loc[i - 1])
    ofi_values.append(ofi)

best_level['OFI'] = ofi_values

# Resample to 1-minute intervals
best_level.set_index('ts_event', inplace=True)
ofi_minutely = best_level['OFI'].resample('1min').sum().reset_index()

# Add symbol column back
ofi_minutely['symbol'] = 'AAPL'

# Preview
ofi_minutely.head()


,ts_event,OFI,symbol
0,2024-10-21 11:54:00+00:00,-405,AAPL
1,2024-10-21 11:55:00+00:00,-1606,AAPL
2,2024-10-21 11:56:00+00:00,-75,AAPL
3,2024-10-21 11:57:00+00:00,-662,AAPL
4,2024-10-21 11:58:00+00:00,257,AAPL


In [5]:
#Multi level OFI
# Reset index just in case
df['ts_event'] = pd.to_datetime(df['ts_event'])

# Filter for AAPL, levels 0–9
aapl_df = df[(df['symbol'] == 'AAPL') & (df['depth'].between(0, 9))].copy()
aapl_df = aapl_df.sort_values('ts_event')
aapl_df.set_index('ts_event', inplace=True)

# Compute OFI for each level based on bid_sz - ask_sz
multi_ofi = {}
for level in range(10):
    bid_col = f'bid_sz_{level:02d}'
    ask_col = f'ask_sz_{level:02d}'

    if bid_col in aapl_df.columns and ask_col in aapl_df.columns:
        ofi_series = (aapl_df[bid_col] - aapl_df[ask_col]).resample('1min').sum()
        multi_ofi[f'ofi_{level}'] = ofi_series
    else:
        print(f"[!] Skipping level {level} — missing {bid_col} or {ask_col}")

# Combine into one DataFrame
multi_ofi_df = pd.DataFrame(multi_ofi).dropna()

# Preview
multi_ofi_df.head()


,ofi_0,ofi_1,ofi_2,ofi_3,ofi_4,ofi_5,ofi_6,ofi_7,ofi_8,ofi_9
ts_event,,,,,,,,,,
2024-10-21 11:54:00+00:00,-2042,7311,3012,-3570,-170,-380,-4218,-354,-971,-6141
2024-10-21 11:55:00+00:00,-6595,-349,4090,-4996,2270,-2621,-2225,61,-9409,-9730
2024-10-21 11:56:00+00:00,-6404,-2941,4187,-375,2292,-5888,1816,-2330,-11292,-1849
2024-10-21 11:57:00+00:00,-18956,-6791,1407,-5608,-4065,939,-6395,-356,-8975,-17460
2024-10-21 11:58:00+00:00,-20449,-483,6724,-6150,-5604,-5275,-6136,-6851,-9082,-16856


In [6]:
#Normalize Multi-Level OFI
# Reset index to access original columns again
df = df.reset_index()

normalized_ofi = multi_ofi_df.copy()

for col in normalized_ofi.columns:
    level = col.split('_')[1]  # Extract level number (e.g., '0')
    level_str = f"{int(level):02d}"  # Format to two digits

    bid_col = f'bid_sz_{level_str}'
    ask_col = f'ask_sz_{level_str}'

    if bid_col in df.columns and ask_col in df.columns:
        avg_depth = (df[bid_col] + df[ask_col]).mean() / 2
        if avg_depth == 0 or pd.isna(avg_depth):
            print(f"[!] Skipping {col} (zero or NaN avg depth)")
            continue
        normalized_ofi[col] = normalized_ofi[col] / avg_depth
    else:
        print(f"[!] Skipping {col} — missing columns: {bid_col}, {ask_col}")

# Preview
normalized_ofi.head()


,ofi_0,ofi_1,ofi_2,ofi_3,ofi_4,ofi_5,ofi_6,ofi_7,ofi_8,ofi_9
ts_event,,,,,,,,,,
2024-10-21 11:54:00+00:00,-9.974068,34.548280,13.146156,-13.951700,-0.697905,-2.333281,-25.587363,-2.000038,-5.550820,-48.345880
2024-10-21 11:55:00+00:00,-32.213017,-1.649207,17.851188,-19.524564,9.319087,-16.093498,-13.497364,0.344639,-53.787506,-76.600783
2024-10-21 11:56:00+00:00,-31.280085,-13.897756,18.274554,-1.465515,9.409404,-36.153574,11.016276,-13.164095,-64.551867,-14.556511
2024-10-21 11:57:00+00:00,-92.589834,-32.091010,6.140983,-21.916284,-16.688144,5.765660,-38.793548,-2.011338,-51.306501,-137.456287
2024-10-21 11:58:00+00:00,-99.882333,-2.282426,29.347528,-24.034441,-23.006239,-32.389624,-37.222394,-38.706958,-51.918177,-132.701213


In [7]:
 #Integrated OFI
 from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Prepare matrix for PCA
X = normalized_ofi.values

# Standardize features (important for PCA)
X_std = StandardScaler().fit_transform(X)

# Apply PCA
pca = PCA(n_components=1)
integrated_ofi = pca.fit_transform(X_std)

# Normalize with L1 norm of the first principal component's weights
w1 = pca.components_[0]
ofi_integrated = (X @ w1) / np.sum(np.abs(w1))

# Add to DataFrame
normalized_ofi['integrated_ofi'] = ofi_integrated

# View result
normalized_ofi[['integrated_ofi']].head()


,integrated_ofi
ts_event,
2024-10-21 11:54:00+00:00,-15.323108
2024-10-21 11:55:00+00:00,-17.854243
2024-10-21 11:56:00+00:00,-9.349377
2024-10-21 11:57:00+00:00,-21.919743
2024-10-21 11:58:00+00:00,-34.090504
